<a href="https://colab.research.google.com/github/Amik-Sen-Fun/FastOnline-SVD-Python-Code/blob/main/Recommeder_Term_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

## Helper function `increment_svd` for future use in the driver code

In [97]:
def increment_svd(U,S,V,A,B):
  d_u = np.shape(U)
  d_a = np.shape(A)
  rankU = d_u[1]
  QP, RP = np.linalg.qr(np.concatenate((U0, A), axis = 1), mode='complete')
  w = rankU+1
  d_qp = np.shape(QP)
  d_rp = np.shape(RP)
  
  if d_qp[1]== rankU:
    P = 0
    Ra = 0
    dim_Ra = 0
  else:
    P = QP[:,w-1:d_qp[1]]
    Ra = RP[w-1:d_rp[0], w-1:d_rp[1]]
    dim_Ra = d_rp[0] - rankU
  
  v = rankU+d_a[1]
  M = RP[0:rankU, w-1:v]
  QQ, RQ = np.linalg.qr(np.concatenate((V, B), axis = 1), mode='complete')
  d_qq = np.shape(QQ)
  d_rq = np.shape(RQ)

  if d_qq[1]== rankU:
    Q = 0
    Rb = 0
    dim_Rb = 0
  else:
    Q = QQ[:,w-1:d_qq[1]]
    Rb = RQ[w-1:d_rq[0], w-1:d_rq[1]]
    dim_Rb = d_rq[0] - rankU

  N = np.matmul(np.transpose(V),B)
  temp0 = np.concatenate((M, Ra), axis = 0)

  if Rb == 0:
    temp1 = N
  else: 
    temp1 = np.concatenate((N, Rb), axis = 0)

  S_augmented = np.matmul(temp0, np.transpose(temp1))
  temp2 = S
  temp3 = np.random.randint(1, size=(dim_Ra, rankU))
  
  S_new = np.concatenate((temp2, temp3), axis = 0) + S_augmented

  updated_svd_u, updated_svd_s, updated_svd_v = np.linalg.svd(S_new, full_matrices=False)

  U_new = np.matmul(np.concatenate((U,P), axis = 1),updated_svd_u)
  S_new = updated_svd_s

  if Q ==0:
    V_new = np.matmul(V,updated_svd_v)
  else:
    V_new = np.matmul(np.concatenate((V,Q), axis = 1),updated_svd_v)

  return U_new, V_new, np.diag(S_new)

## The finally implemented example with the help of the `increment_svd` function

In [98]:
# Keep m > n
m = 4 # number of users
n = 2 # number of products
# Generate the X random matrix
X = np.random.randint(6, size=(m, n))
X

array([[5, 5],
       [3, 2],
       [5, 2],
       [2, 3]])

In [99]:
# Get the SVD for the current matrix
u, s, v = np.linalg.svd(X, full_matrices=False)
# Transforming s into a diagonal matrix for further calculation
s = np.diag(s)
print(u)
print(v)
print(s)

[[-0.70053799  0.37220578]
 [-0.35799888 -0.1512169 ]
 [-0.51356624 -0.75141533]
 [-0.34253911  0.52342268]]
[[-0.78039401 -0.62528808]
 [-0.62528808  0.78039401]]
[[10.03287557  0.        ]
 [ 0.          2.08360453]]


In [100]:
# Suppose we get a new streaming data 
w = m+1
B = np.random.randint(6, size=(n, 1))
A = np.random.randint(1, size=(w, 1))
A[w-1][0] = 1

In [101]:
temp = np.random.randint(1, size=(1, n))

U0 = np.concatenate((u, temp), axis = 0)
X0 = np.concatenate((X, temp), axis = 0)

# Function to perform the Rank-1 update
updated_u, updated_v, updated_s = increment_svd(U0,s,v,A,B)

In [103]:
# Checing the final error with fast SVD
error_part1 = np.add(X0,np.matmul(A,np.transpose(B)))
error_part2 = np.matmul(np.matmul(updated_u,updated_s),np.transpose(updated_v))
error = np.subtract(error_part1,error_part2)
error_value = np.linalg.norm(error,2)
print(f"The value of error is {error_value}")

The value of error is 0.6286379941349529
